In [453]:
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.special import sph_harm, jn
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab

mlab.init_notebook()

s = mlab.test_plot3d()
mlab.show()

Notebook initialized with ipy backend.


In [454]:
# 積分する関数を定義
def integrand(theta, phi, l, m, lp, mp):
    Ylm = sph_harm(m, l, phi, theta)
    Ylmp = sph_harm(mp, lp, phi, theta).conj()
    return np.real(Ylm * Ylmp) * np.sin(theta)

In [455]:
# 積分範囲
theta_range = (0, np.pi)
phi_range = (0, 2 * np.pi)

# 球面調和関数のパラメータ
l, m = 1, 0
lp, mp = 0, 0

# 積分計算
integral, error = dblquad(
    integrand,
    phi_range[0],
    phi_range[1],
    lambda x: theta_range[0],
    lambda x: theta_range[1],
    args=(l, m, lp, mp),
)

print(f"Integral = {integral}, Error = {error}")

Integral = 6.724179670223125e-17, Error = 1.5161163041736918e-15


In [456]:
import numpy as np
import plotly.graph_objects as go
from scipy.special import sph_harm

In [457]:
import numpy as np
from scipy.special import sph_harm, spherical_jn, legendre
import plotly.graph_objects as go


def approximate_plane_wave(k, r, theta, phi, l_max):
    approx = np.zeros_like(theta, dtype=np.complex_)
    # 球面調和関数での展開
    for l in range(l_max + 1):
        for m in range(-l, l + 1):
            # 展開係数の計算
            coeff = (
                4 * np.pi * 1j**l * sph_harm(m, l, 0, 0).conj() * spherical_jn(l, k * r)
            )
            # 球面調和関数の加算
            approx += coeff * sph_harm(m, l, phi, theta)
    return approx


def approximate_plane_wave2(k, r, theta, phi, l_max):
    approx = np.zeros_like(theta, dtype=np.complex_)
    # 球面調和関数での展開
    for l in range(l_max + 1):
        # 展開係数の計算
        coeff = (2 * l + 1) * 1j**l * spherical_jn(l, k * r)
        # 球面調和関数の加算
        approx += coeff * legendre(l)(np.cos(theta))
    return approx

In [458]:
# パラメータ設定
k = 1.0  # 波数
l_max = 30  # 展開するlの最大値
z = np.linspace(-20 * np.pi, 20 * np.pi, 1000)
r = np.abs(z)
theta = np.arccos(z / r)

# z=0での断面における平面波の近似を計算
phi = np.zeros(z.shape)  # z=0ではphiは0とする
approx = approximate_plane_wave(k, r, theta, phi, l_max)

# 絶対値を取ることで、実数の値に変換
real_approx = approx.real
imag_approx = approx.imag

# Plotlyで2次元ヒートマップとしてプロット
fig = go.Figure()
fig.add_trace(go.Scatter(x=z, y=real_approx))

fig.update_layout(
    xaxis_title="z",
    yaxis_title="plane wave approximation",
    autosize=False,
    width=700,
    height=500,
)
fig.show()
fig = go.Figure()
fig.add_trace(go.Scatter(x=z, y=imag_approx))

fig.update_layout(
    xaxis_title="z",
    yaxis_title="plane wave approximation",
    autosize=False,
    width=700,
    height=500,
)
fig.show()

In [459]:
x = np.linspace(-10 * np.pi, 10 * np.pi, 100)
z = np.linspace(-10 * np.pi, 10 * np.pi, 100)
X, Z = np.meshgrid(x, z)
r = np.sqrt(X**2 + Z**2)
phi = np.zeros(X.shape)
theta = np.arccos(Z / r)
approx = approximate_plane_wave(k, r, theta, phi, l_max)
approx_real = approx.real

In [460]:
fig = go.Figure()
fig.add_trace(
    go.Surface(
        x=X,
        y=np.zeros(X.shape),
        z=Z,
        surfacecolor=approx_real,
        colorscale="Viridis",
        showscale=True,
    )
)
# レイアウトの設定
fig.update_layout(title="3D Surface Plot", autosize=False, width=500, height=500)
fig.show()